In [1]:
# https://stackoverflow.com/questions/46288847/how-to-suppress-pip-upgrade-warning
# --disable-pip-version-check --root-user-action=ignore
!pip config set global.disable-pip-version-check true
!pip config set global.root-user-action ignore


import os
os.environ['CUDA_MODULE_LOADING']='LAZY'

!mkdir -p /kaggle/tmp/libs

# # upgrade pytorch to 1.13.1
# !pip install /kaggle/input/torch1131-cu116/torch1131/torch-1.13.1+cu116-cp37-cp37m-linux_x86_64.whl
# !pip install /kaggle/input/torch1131-cu116/torch1131/torchvision-0.14.1+cu116-cp37-cp37m-linux_x86_64.whl

# timm 0.8.11.dev0
!pip uninstall -y timm
!cp -r /kaggle/input/kaggle-rsna-pkgs/timm /kaggle/tmp/libs
%cd /kaggle/tmp/libs/timm
!pip install -e .
%cd /kaggle/working

try: 
    import torch2trt
except:
    !pip install /kaggle/input/torch-tensorrt-pkg/nvidia_pyindex-1.0.9-py3-none-any.whl
    !mkdir -p /tmp/pip/cache/
    !cp /kaggle/input/torch-tensorrt-pkg/nvidia-cublas-cu11-2022.4.8.xyz /tmp/pip/cache/nvidia-cublas-cu11-2022.4.8.tar.gz
    !cp /kaggle/input/torch-tensorrt-pkg/nvidia-cuda-runtime-cu11-2022.4.25.xyz /tmp/pip/cache/nvidia-cuda-runtime-cu11-2022.4.25.tar.gz
    !cp /kaggle/input/torch-tensorrt-pkg/nvidia-cudnn-cu11-2022.5.19.xyz /tmp/pip/cache/nvidia-cudnn-cu11-2022.5.19.tar.gz
    !cp /kaggle/input/torch-tensorrt-pkg/nvidia_cublas_cu117-11.10.1.25-py3-none-manylinux1_x86_64.whl /tmp/pip/cache/
    !cp /kaggle/input/torch-tensorrt-pkg/nvidia_cuda_runtime_cu117-11.7.60-py3-none-manylinux1_x86_64.whl /tmp/pip/cache/
    !cp /kaggle/input/torch-tensorrt-pkg/nvidia_cudnn_cu116-8.4.0.27-py3-none-manylinux1_x86_64.whl /tmp/pip/cache/
    !cp /kaggle/input/torch-tensorrt-pkg/nvidia_tensorrt-8.4.3.1-cp37-none-linux_x86_64.whl /tmp/pip/cache/
    !pip install --no-index --find-links /tmp/pip/cache/ nvidia_tensorrt
# #     !pip install /kaggle/input/torch-tensorrt-pkg/torch_tensorrt-1.2.0-cp37-cp37m-linux_x86_64.whl

    
    !cp -r /kaggle/input/kaggle-rsna-pkgs/torch2trt /kaggle/tmp/libs
    %cd /kaggle/tmp/libs/torch2trt
    !python setup.py install
    !pip install -e .
#     !cmake -B build . && cmake --build build --target install && ldconfig
    %cd /kaggle/working/

try:
    import dicomsdl
except:
    !pip install /kaggle/input/kaggle-rsna-pkgs/pylibjpeg-1.4.0-py3-none-any.whl
    !pip install /kaggle/input/kaggle-rsna-pkgs/python_gdcm-3.0.21-cp37-cp37m-manylinux_2_17_x86_64.manylinux2014_x86_64.whl
    !pip install /kaggle/input/kaggle-rsna-pkgs/dicomsdl-0.109.1-cp37-cp37m-manylinux_2_12_x86_64.manylinux2010_x86_64.whl

try:
    import dali
except:
    !pip install /kaggle/input/kaggle-rsna-pkgs/nvidia_dali_nightly_cuda110-1.23.0.dev20230210-7260679-py3-none-manylinux2014_x86_64.whl

# try:
#     import nvjpeg2k
# except:
#     # For NVJPEG2k
#     !cp /kaggle/input/kaggle-rsna-pkgs/nvjpeg2k.so ./

print('Import done!')

Writing to /root/.config/pip/pip.conf
Writing to /root/.config/pip/pip.conf
Found existing installation: timm 0.6.12
Uninstalling timm-0.6.12:
  Successfully uninstalled timm-0.6.12
/kaggle/tmp/libs/timm
Obtaining file:///kaggle/tmp/libs/timm
  Preparing metadata (setup.py) ... - done
  Running setup.py develop for timm
/kaggle/working
Processing /kaggle/input/torch-tensorrt-pkg/nvidia_pyindex-1.0.9-py3-none-any.whl
Looking in links: /tmp/pip/cache/
Processing /tmp/pip/cache/nvidia_tensorrt-8.4.3.1-cp37-none-linux_x86_64.whl
Processing /tmp/pip/cache/nvidia-cudnn-cu11-2022.5.19.tar.gz
  Preparing metadata (setup.py) ... - done
Processing /tmp/pip/cache/nvidia-cublas-cu11-2022.4.8.tar.gz
  Preparing metadata (setup.py) ... - done
Processing /tmp/pip/cache/nvidia-cuda-runtime-cu11-2022.4.25.tar.gz
  Preparing metadata (setup.py) ... - done
Processing /tmp/pip/cache/nvidia_cublas_cu117-11.10.1.25-py3-none-manylinux1_x86_64.whl
Processing /tmp/pip/cache/nvidia_cuda_runtime_cu117-11

In [2]:
import sys
sys.path.append('/kaggle/tmp/libs/timm')
sys.path.append('/opt/conda/lib/python3.7/site-packages/torch2trt-0.4.0-py3.7.egg')
sys.path.append('/kaggle/tmp/libs/torch2trt')
import timm
import gc
print('Timm version:', timm.__version__)

Timm version: 0.8.11dev0


In [3]:
%%writefile metrics.py


import matplotlib.pyplot as plt
import numpy as np
import sklearn
import torch
from sklearn import metrics
from sklearn.metrics import (auc, confusion_matrix,
                             precision_recall_fscore_support, roc_curve)


def pfbeta_np(gts, preds, beta=1):
    preds = preds.clip(0, 1.)
    y_true_count = gts.sum()
    ctp = preds[gts == 1].sum()
    cfp = preds[gts == 0].sum()
    beta_squared = beta * beta
    c_precision = ctp / (ctp + cfp)
    c_recall = ctp / y_true_count
    if (c_precision > 0 and c_recall > 0):
        ret = (1 + beta_squared) * (c_precision * c_recall) / (
            beta_squared * c_precision + c_recall)
        return ret
    else:
        return 0.0


def _compute_fbeta(precision, recall, beta=1.0):
    return (1 + beta**2) * precision * recall / (
        (beta**2) * precision + recall)


def compute_usual_metrics(gts, preds, beta=1.0, sample_weights=None):
    """Binary prediction only."""
    cfm = confusion_matrix(gts,
                           preds,
                           labels=[0, 1],
                           sample_weight=sample_weights)

    tn, fp, fn, tp = cfm.ravel()
    acc = (tp + tn) / (tn + fp + fn + tp)
    precision = tp / (tp + fp)
    recall = tp / (tp + fn)
    fbeta = _compute_fbeta(precision, recall, beta=beta)
    # frr = fp / (fp + tn)
    # far = fn / (fn + tp)  # 1 - recall
    # bacc_beta = _compute_fbeta(1 - frr, 1 - far, beta=beta)
    return {
        'acc': acc,
        'precision': precision,
        'recall': recall,
        'fbeta': fbeta,
        # 'bacc_beta': bacc_beta,
        # 'frr': frr,
        # 'far': far,
    }


def compute_metrics_over_thresholds(preds,
                                    gts,
                                    thresholds=np.linspace(0, 1, 101),
                                    eps=1e-3):
    f1scores = []
    precisions = []
    recalls = []
    for t in thresholds:
        predict = (preds > t).astype(np.float32)

        tp = ((predict >= 0.5) & (gts >= 0.5)).sum()
        fp = ((predict >= 0.5) & (gts < 0.5)).sum()
        fn = ((predict < 0.5) & (gts >= 0.5)).sum()

        r = tp / (tp + fn + eps)
        p = tp / (tp + fp + eps)
        f1 = 2 * r * p / (r + p + eps)
        f1scores.append(f1)
        precisions.append(p)
        recalls.append(r)
    f1scores = np.array(f1scores)
    precisions = np.array(precisions)
    recalls = np.array(recalls)
    return f1scores, precisions, recalls, thresholds


def compute_best_metrics(cancer_p, cancer_t):

    fpr, tpr, thresholds = metrics.roc_curve(cancer_t, cancer_p)
    auc = metrics.auc(fpr, tpr)

    f1scores, precisions, recalls, thresholds = compute_metrics_over_thresholds(
        cancer_p, cancer_t)
    i = f1scores.argmax()
    f1score, precision, recall, threshold = f1scores[i], precisions[
        i], recalls[i], thresholds[i]

    specificity = ((cancer_p < threshold) &
                   ((cancer_t <= 0.5))).sum() / (cancer_t <= 0.5).sum()
    sensitivity = ((cancer_p >= threshold) &
                   ((cancer_t >= 0.5))).sum() / (cancer_t >= 0.5).sum()

    return {
        'auc': auc,
        'threshold': threshold,
        'f1score': f1score,
        'precision': precision,
        'recall': recall,
        'sensitivity': sensitivity,
        'specificity': specificity,
    }


def print_all_metric(valid_df):

    print(
        f'{"    ": <16}    \tauc      @th     f1      | 	prec    recall  | 	sens    spec '
    )
    for site_id in [0, 1, 2]:
        if site_id > 0:
            site_df = valid_df[valid_df.site_id == site_id].reset_index(
                drop=True)
        else:
            site_df = valid_df
        # ---

        gb = site_df
        m = compute_best_metrics(gb.cancer_p, gb.cancer_t)
        text = f'{"single image": <16} [{site_id}]'
        text += f'\t{m["auc"]:0.5f}'
        text += f'\t{m["threshold"]:0.5f}'
        text += f'\t{m["f1score"]:0.5f} | '
        text += f'\t{m["precision"]:0.5f}'
        text += f'\t{m["recall"]:0.5f} | '
        text += f'\t{m["sensitivity"]:0.5f}'
        text += f'\t{m["specificity"]:0.5f}'
        #text += '\n'
        print(text)

        # ---

        gb = site_df[['patient_id', 'laterality', 'cancer_t',
                      'cancer_p']].groupby(['patient_id',
                                            'laterality']).mean()
        m = compute_best_metrics(gb.cancer_p, gb.cancer_t)
        text = f'{"grouby mean()": <16} [{site_id}]'
        text += f'\t{m["auc"]:0.5f}'
        text += f'\t{m["threshold"]:0.5f}'
        text += f'\t{m["f1score"]:0.5f} | '
        text += f'\t{m["precision"]:0.5f}'
        text += f'\t{m["recall"]:0.5f} | '
        text += f'\t{m["sensitivity"]:0.5f}'
        text += f'\t{m["specificity"]:0.5f}'
        #text += '\n'
        print(text)

        # ---
        gb = site_df[['patient_id', 'laterality', 'cancer_t',
                      'cancer_p']].groupby(['patient_id', 'laterality']).max()
        m = compute_best_metrics(gb.cancer_p, gb.cancer_t)
        text = f'{"grouby max()": <16} [{site_id}]'
        text += f'\t{m["auc"]:0.5f}'
        text += f'\t{m["threshold"]:0.5f}'
        text += f'\t{m["f1score"]:0.5f} | '
        text += f'\t{m["precision"]:0.5f}'
        text += f'\t{m["recall"]:0.5f} | '
        text += f'\t{m["sensitivity"]:0.5f}'
        text += f'\t{m["specificity"]:0.5f}'
        #text += '\n'
        print(text)
        print(f'--------------\n')


def compute_all(df, plot_save_path):
    print(f'Saving plot to {plot_save_path}')
    df['cancer_p'] = df['preds']
    df['cancer_t'] = df['targets']
    print_all_metric(df)

    gb = df[['site_id', 'patient_id', 'laterality', 'cancer_t',
             'cancer_p']].groupby(['patient_id', 'laterality']).mean()
    gb.loc[:, 'cancer_t'] = gb.cancer_t.astype(int)
    m = compute_best_metrics(gb.cancer_p, gb.cancer_t)
    text = f'{"grouby mean()": <16}'
    text += f'\t{m["auc"]:0.5f}'
    text += f'\t{m["threshold"]:0.5f}'
    text += f'\t{m["f1score"]:0.5f} | '
    text += f'\t{m["precision"]:0.5f}'
    text += f'\t{m["recall"]:0.5f} | '
    text += f'\t{m["sensitivity"]:0.5f}'
    text += f'\t{m["specificity"]:0.5f}'
    text += '\n'
    print(text)

    pfbeta = pfbeta_np(gb.cancer_t.values, gb.cancer_p.values, beta=1)
    print('PROBABILITY-FBETA:', pfbeta)

    plot_pr_curve(gb, plot_save_path)


def plot_pr_curve(df, plot_save_path):
    f1scores, precisions, recalls, thresholds = compute_metrics_over_thresholds(
        df.cancer_p, df.cancer_t)
    i = f1scores.argmax()
    f1score_max, precision_max, recall_max, threshold_max = f1scores[
        i], precisions[i], recalls[i], thresholds[i]
    print(
        f'f1score_max = {f1score_max}, precision_max = {precision_max}, recall_max = {recall_max}, threshold_max = {threshold_max}'
    )

    _, axs = plt.subplots(2, 2, figsize=(20, 15))

    ############################################################################
    ### PRECISION-RECALL CURVE
    f_scores = [0.2, 0.3, 0.4, 0.5, 0.6, 0.7,
                0.8]  #np.linspace(0.2, 0.8, num=8)
    for f_score in f_scores:
        x = np.linspace(0.01, 1)
        y = f_score * x / (2 * x - f_score)
        (l, ) = axs[0, 0].plot(x[y >= 0], y[y >= 0], color="gray", alpha=0.2)
        axs[0, 0].annotate("f1={0:0.1f}".format(f_score),
                           xy=(0.9, y[45] + 0.02))
    axs[0, 0].plot([0, 1], [0, 1], color="gray", alpha=0.2)

    # overall
    precision, recall, threshold = metrics.precision_recall_curve(
        df.cancer_t, df.cancer_p)
    auc = metrics.auc(recall, precision)
    axs[0, 0].plot(recall, precision)
    s = axs[0, 0].scatter(recall[:-1], precision[:-1], c=threshold, cmap='hsv')
    axs[0, 0].scatter(recall_max, precision_max, s=30, c='k')

    # for each site
    precision, recall, threshold = metrics.precision_recall_curve(
        df.cancer_t[df.site_id == 1], df.cancer_p[df.site_id == 1])
    axs[0, 0].plot(recall, precision, '--', label='site_id=1')
    precision, recall, threshold = metrics.precision_recall_curve(
        df.cancer_t[df.site_id == 2], df.cancer_p[df.site_id == 2])
    axs[0, 0].plot(recall, precision, '--', label='site_id=2')

    axs[0, 0].set_xlim([0.0, 1.0])
    axs[0, 0].set_ylim([0.0, 1.05])

    text = ''
    text += f'MAX f1score {f1score_max: 0.5f} @ th = {threshold_max: 0.5f}\n'
    text += f'prec {precision_max: 0.5f}, recall {recall_max: 0.5f}, pr-auc {auc: 0.5f}\n'

    axs[0, 0].legend()
    axs[0, 0].set_title(text)
    plt.colorbar(s, ax=axs[0, 0], label='threshold')
    axs[0, 0].set_xlabel('recall')
    axs[0, 0].set_ylabel('precision')

    ############################################################################
    # HISTOGRAM
    spacing = 51

    for site_type in [0, 1, 2]:
        if site_type == 0:
            ax = axs[0, 1]
            sub_df = df
            title = 'All site'
        elif site_type == 1:
            ax = axs[1, 0]
            sub_df = df[df.site_id == site_type].reset_index(drop=True)
            title = 'Site 1'
        elif site_type == 2:
            ax = axs[1, 1]
            sub_df = df[df.site_id == site_type].reset_index(drop=True)
            title = 'Site 2'

        cancer_p = sub_df.cancer_p
        cancer_t = sub_df.cancer_t
        cancer_t = cancer_t.astype(int)
        pos, bin = np.histogram(cancer_p[cancer_t == 1],
                                np.linspace(0, 1, spacing))
        neg, bin = np.histogram(cancer_p[cancer_t == 0],
                                np.linspace(0, 1, spacing))
        pos = pos / (cancer_t == 1).sum()
        neg = neg / (cancer_t == 0).sum()
        # plt.plot(bin[1:],neg, alpha=1)
        # plt.plot(bin[1:],pos, alpha=1)
        bin = (bin[1:] + bin[:-1]) / 2
        ax.bar(bin, neg, width=1 / spacing, label='neg', alpha=0.5)
        ax.bar(bin, pos, width=1 / spacing, label='pos', alpha=0.5)
        ax.legend()
        ax.set_title(title)

    # plt.show()
    plt.savefig(plot_save_path)


def _compute_metrics(gts,
                     preds,
                     sample_weights=None,
                     thres_range=(0, 1, 0.01),
                     sort_by='pfbeta'):
    if isinstance(gts, torch.Tensor):
        gts = gts.cpu().numpy()
    if isinstance(preds, torch.Tensor):
        preds = preds.cpu().numpy()
    assert isinstance(gts, np.ndarray) and isinstance(preds, np.ndarray)
    assert len(preds) == len(gts)

    # Probabilistic-fbeta
    pfbeta = pfbeta_np(gts, preds, beta=1.0)
    # AUC
    fpr, tpr, _thresholds = sklearn.metrics.roc_curve(gts, preds, pos_label=1)
    auc = sklearn.metrics.auc(fpr, tpr)

    # PR-AUC
    precisions, recalls, _thresholds = sklearn.metrics.precision_recall_curve(
        gts, preds)
    pr_auc = sklearn.metrics.auc(recalls, precisions)

    ##### METRICS FOR CATEGORICAL PREDICTION #####
    # PER THRESHOLD METRIC
    per_thres_metrics = []
    for thres in np.arange(*thres_range):
        bin_preds = (preds > thres).astype(np.uint8)
        metric_at_thres = compute_usual_metrics(gts, bin_preds, beta=1.0)
        pfbeta_at_thres = pfbeta_np(gts, bin_preds, beta=1.0)
        metric_at_thres['pfbeta'] = pfbeta_at_thres

        if sample_weights is not None:
            w_metric_at_thres = compute_usual_metrics(gts, bin_preds, beta=1.0)
            w_metric_at_thres = {
                f'w_{k}': v
                for k, v in w_metric_at_thres.items()
            }
            metric_at_thres.update(w_metric_at_thres)
        per_thres_metrics.append((thres, metric_at_thres))

    per_thres_metrics.sort(key=lambda x: x[1][sort_by], reverse=True)

    # handle multiple thresholds with same scores
    top_score = per_thres_metrics[0][1][sort_by]
    same_scores = []
    for j, (thres, metric_at_thres) in enumerate(per_thres_metrics):
        if metric_at_thres[sort_by] == top_score:
            same_scores.append(abs(thres - 0.5))
        else:
            assert metric_at_thres[sort_by] < top_score
            break
    if len(same_scores) == 1:
        best_thres, best_metric = per_thres_metrics[0]
    else:
        # the nearer 0.5 threshold is --> better
        best_idx = np.argmin(np.array(same_scores))
        best_thres, best_metric = per_thres_metrics[best_idx]

    # best thres, best results, all results
    return {
        'best_thres': best_thres,
        'best_metric': best_metric,
        'all_metrics': per_thres_metrics,
        'pfbeta': pfbeta,
        'auc': auc,
        'prauc': pr_auc,
    }


def compute_metrics(df,
                    plot_save_path='plot.png',
                    thres_range=(0, 1, 0.01),
                    sort_by='pfbeta',
                    additional_info=False):
    ori_df = df[[
        'site_id', 'patient_id', 'laterality', 'cancer', 'preds', 'targets'
    ]]
    all_metrics = {}

    reducer_single = lambda df: df
    reducer_gbmean = lambda df: df.groupby(['patient_id', 'laterality']).mean()
    reducer_gbmax = lambda df: df.groupby(['patient_id', 'laterality']).mean()
    reducer_gbmean_site1 = lambda df: df[df.site_id == 1].reset_index(
        drop=True).groupby(['patient_id', 'laterality']).mean()
    reducer_gbmean_site2 = lambda df: df[df.site_id == 2].reset_index(
        drop=True).groupby(['patient_id', 'laterality']).mean()

    reducers = {
        'single': reducer_single,
        'gbmean': reducer_gbmean,
        'gbmean_site1': reducer_gbmean_site1,
        'gbmean_site2': reducer_gbmean_site2,
        'gbmax': reducer_gbmax,
    }

    for reducer_name, reducer in reducers.items():
        df = reducer(ori_df.copy())
        preds = df['preds'].to_numpy()
        gts = df['targets'].to_numpy()
        # mean_sample_weights = mean_df['sample_weights']
        _metrics = _compute_metrics(gts, preds, None, thres_range, sort_by)
        all_metrics[f'{reducer_name}_best_thres'] = _metrics['best_thres']
        all_metrics.update({
            f'{reducer_name}_best_{k}': v
            for k, v in _metrics['best_metric'].items()
        })
        all_metrics[f'{reducer_name}_pfbeta'] = _metrics['pfbeta']
        all_metrics[f'{reducer_name}_auc'] = _metrics['auc']
        all_metrics[f'{reducer_name}_prauc'] = _metrics['prauc']

    # rank 0 only
    if additional_info:
        compute_all(ori_df, plot_save_path)
    return all_metrics

Writing metrics.py


## ROI extraction:

YOLOX + (Otsu find contours) fallback

In [4]:
%%writefile roi_extract.py


import os

os.environ['CUDA_MODULE_LOADING'] = 'LAZY'

import sys

import cv2
import numpy as np
import torch
import torchvision

sys.path.append('/kaggle/tmp/libs/')
from torch2trt import TRTModule
from torch.nn import functional as F

_TORCH_VER = [int(x) for x in torch.__version__.split(".")[:2]]
_TORCH11X = (_TORCH_VER >= [1, 10])


def meshgrid(*tensors):
    if _TORCH11X:
        return torch.meshgrid(*tensors, indexing="ij")
    else:
        return torch.meshgrid(*tensors)


def extract_roi_otsu(img, gkernel=(5, 5)):
    """WARNING: this function modify input image inplace."""
    ori_h, ori_w = img.shape[:2]
    # clip percentile: implant, white lines
    upper = np.percentile(img, 95)
    img[img > upper] = np.min(img)
    # Gaussian filtering to reduce noise (optional)
    if gkernel is not None:
        img = cv2.GaussianBlur(img, gkernel, 0)
    _, img_bin = cv2.threshold(img, 0, 255,
                               cv2.THRESH_BINARY + cv2.THRESH_OTSU)
    # dilation to improve contours connectivity
    element = cv2.getStructuringElement(cv2.MORPH_RECT, (3, 3), (-1, -1))
    img_bin = cv2.dilate(img_bin, element)
    cnts, _ = cv2.findContours(img_bin, cv2.RETR_EXTERNAL,
                               cv2.CHAIN_APPROX_SIMPLE)
    if len(cnts) == 0:
        return None, None, None
    areas = np.array([cv2.contourArea(cnt) for cnt in cnts])
    select_idx = np.argmax(areas)
    cnt = cnts[select_idx]
    area_pct = areas[select_idx] / (img.shape[0] * img.shape[1])
    x0, y0, w, h = cv2.boundingRect(cnt)
    # min-max for safety only
    # x0, y0, x1, y1
    x1 = min(max(int(x0 + w), 0), ori_w)
    y1 = min(max(int(y0 + h), 0), ori_h)
    x0 = min(max(int(x0), 0), ori_w)
    y0 = min(max(int(y0), 0), ori_h)
    return [x0, y0, x1, y1], area_pct, None


class RoiExtractor:

    def __init__(self,
                 engine_path,
                 input_size,
                 num_classes,
                 conf_thres=0.5,
                 nms_thres=0.9,
                 class_agnostic=False,
                 area_pct_thres=0.04,
                 hw=None,
                 strides=None,
                 exp=None):
        self.input_size = input_size
        self.input_h, self.input_w = input_size
        self.num_classes = num_classes
        self.conf_thres = conf_thres
        self.nms_thres = nms_thres
        self.class_agnostic = class_agnostic
        self.area_pct_thres = area_pct_thres

        model = TRTModule()
        model.load_state_dict(torch.load(engine_path))
        self.model = model
        if hw is None or strides is None:
            assert exp is not None
            self._set_meta(exp)
        else:
            # [torch.Size([80, 80]), torch.Size([40, 40]), torch.Size([20, 20])]
            self.hw = hw
            # [8, 16, 32]
            self.strides = strides

    def _set_meta(self, exp):
        assert exp is not None
        print("Start probing model metadata..")
        # dummy infer
        torch_model = exp.get_model().cuda().eval()
        _dummy = torch.ones(1, 3, exp.test_size[0], exp.test_size[1]).cuda()
        torch_model(_dummy)
        # set attributes
        self.hw = torch_model.head.hw
        self.strides = torch_model.head.strides
        # cleanup
        del torch_model, _dummy
        import gc
        gc.collect()
        torch.cuda.empty_cache()
        print('Done probbing model metadata..')

    def decode_outputs(self, outputs):
        dtype = outputs.type()
        grids = []
        strides = []
        for (hsize, wsize), stride in zip(self.hw, self.strides):
            yv, xv = meshgrid([torch.arange(hsize), torch.arange(wsize)])
            grid = torch.stack((xv, yv), 2).view(1, -1, 2)
            grids.append(grid)
            shape = grid.shape[:2]
            strides.append(torch.full((*shape, 1), stride))

        grids = torch.cat(grids, dim=1).type(dtype)
        strides = torch.cat(strides, dim=1).type(dtype)

        outputs = torch.cat(
            [(outputs[..., 0:2] + grids) * strides,
             torch.exp(outputs[..., 2:4]) * strides, outputs[..., 4:]],
            dim=-1)
        return outputs

    def post_process(self,
                     pred,
                     conf_thres=0.5,
                     nms_thres=0.9,
                     class_agnostic=False):
        box_corner = pred.new(pred.shape)
        box_corner[:, :, 0] = pred[:, :, 0] - pred[:, :, 2] / 2
        box_corner[:, :, 1] = pred[:, :, 1] - pred[:, :, 3] / 2
        box_corner[:, :, 2] = pred[:, :, 0] + pred[:, :, 2] / 2
        box_corner[:, :, 3] = pred[:, :, 1] + pred[:, :, 3] / 2
        pred[:, :, :4] = box_corner[:, :, :4]

        output = [None for _ in range(len(pred))]
        for i, image_pred in enumerate(pred):

            # If none are remaining => process next image
            if not image_pred.size(0):
                continue
            # Get score and class with highest confidence
            class_conf, class_pred = torch.max(image_pred[:, 5:5 +
                                                          self.num_classes],
                                               1,
                                               keepdim=True)

            conf_mask = (image_pred[:, 4] * class_conf.squeeze() >=
                         conf_thres).squeeze()
            # Detections ordered as (x1, y1, x2, y2, obj_conf, class_conf, class_pred)
            detections = torch.cat(
                (image_pred[:, :5], class_conf, class_pred.float()), 1)
            detections = detections[conf_mask]
            if not detections.size(0):
                continue

            if class_agnostic:
                nms_out_index = torchvision.ops.nms(
                    detections[:, :4],
                    detections[:, 4] * detections[:, 5],
                    nms_thres,
                )
            else:
                nms_out_index = torchvision.ops.batched_nms(
                    detections[:, :4],
                    detections[:, 4] * detections[:, 5],
                    detections[:, 6],
                    nms_thres,
                )
            detections = detections[nms_out_index]
            if output[i] is None:
                output[i] = detections
            else:
                output[i] = torch.cat((output[i], detections))
        return output

    def preprocess_single(self, img: torch.Tensor):
        ori_h = img.size(0)
        ori_w = img.size(1)
        ratio = min(self.input_h / ori_h, self.input_w / ori_w)
        # resize
        resized_img = F.interpolate(img.view(1, 1, ori_h, ori_w),
                                    mode="bilinear",
                                    scale_factor=ratio,
                                    recompute_scale_factor=True)[0, 0]
        # padding
        padded_img = torch.full((self.input_h, self.input_w),
                                114,
                                dtype=resized_img.dtype,
                                device='cuda')
        padded_img[:resized_img.size(0), :resized_img.size(1)] = resized_img
        padded_img = padded_img.unsqueeze(-1).expand(-1, -1, 3)
        # HWC --> CHW
        padded_img = padded_img.permute(2, 0, 1)
        padded_img = padded_img.float()
        return padded_img, resized_img, ratio, ori_h, ori_w

    def detect_single(self, img):
        padded_img, resized_img, ratio, ori_h, ori_w = self.preprocess_single(
            img)
        padded_img = padded_img.unsqueeze(0)
        output = self.model(padded_img)
        output = self.decode_outputs(output)
        # x0, y0, x1, y1, box_conf, cls_conf, cls_id
        output = self.post_process(output, self.conf_thres, self.nms_thres)[0]
        if output is not None:
            output[:, :4] = output[:, :4] / ratio
            # re-compute: conf = box_conf * cls_conf
            output[:, 4] = output[:, 4] * output[:, 5]
            # select box with highest confident
            output = output[output[:, 4].argmax()]
            x0 = min(max(int(output[0]), 0), ori_w)
            y0 = min(max(int(output[1]), 0), ori_h)
            x1 = min(max(int(output[2]), 0), ori_w)
            y1 = min(max(int(output[3]), 0), ori_h)
            area_pct = (x1 - x0) * (y1 - y0) / (ori_h * ori_w)
            if area_pct >= self.area_pct_thres:
                # xyxy, area_pct, conf
                return [x0, y0, x1, y1], area_pct, output[4]

        # if YOLOX fail, try Otsu thresholding + find contours
        xyxy, area_pct, _ = extract_roi_otsu(
            resized_img.to(torch.uint8).cpu().numpy())
        # if both fail, use full frame
        if xyxy is not None:
            if area_pct >= self.area_pct_thres:
                print('ROI detection: using Otsu.')
                x0, y0, x1, y1 = xyxy
                x0 = min(max(int(x0 / ratio), 0), ori_w)
                y0 = min(max(int(y0 / ratio), 0), ori_h)
                x1 = min(max(int(x1 / ratio), 0), ori_w)
                y1 = min(max(int(y1 / ratio), 0), ori_h)
                return [x0, y0, x1, y1], area_pct, None
        print('ROI detection: both fail.')
        return None, area_pct, None
            

Writing roi_extract.py


In [5]:
%%writefile convert_model.py


import os
import sys

import cv2
import numpy as np
import tensorrt as trt
import torch
from torch import nn
from torch2trt import torch2trt as torch2trt

# sys.path.append('/home/dangnh36/projects/.comp/rsna/src/pytorch-image-models/')
from timm.data import resolve_data_config
from timm.models import create_model

MODEL_NAME = 'tf_efficientnet_b4.ns_jft_in1k'
MODEL_CKPTS = [
#     '/kaggle/input/kaggle-rsna-ckpts/fold0_convnext-small_exp4_ratio8_ep11.pth.tar',
    '/kaggle/input/kaggle-rsna-ckpts/fold0_effb4_ep3.tar'
]
NUM_CLASSES = 2
IN_CHANS = 3
MEAN = np.array([0.485, 0.456, 0.406]) * 255
STD = np.array([0.229, 0.224, 0.225]) * 255
GLOBAL_POOL = 'avg'
BATCH_SIZE = 4


class KFoldEnsembleModel(nn.Module):

    def __init__(self, model_name, num_classes, in_chans, ckpt_paths):
        super(KFoldEnsembleModel, self).__init__()
        fmodels = []
        for i, ckpt_path in enumerate(ckpt_paths):
            fmodel = create_model(
                model_name,
                num_classes=num_classes,
                in_chans=in_chans,
                pretrained=False,
                checkpoint_path=ckpt_path,
                global_pool=GLOBAL_POOL,
            ).eval()
            # print(fmodel)
            data_config = resolve_data_config({}, model=fmodel)
            print('Data config:', data_config)
            fmodels.append(fmodel)
        self.fmodels = nn.ModuleList(fmodels)

        self.register_buffer('mean',
                             torch.FloatTensor(MEAN).reshape(1, 3, 1, 1))
        self.register_buffer('std', torch.FloatTensor(STD).reshape(1, 3, 1, 1))

    def forward(self, x):
#         x = x.sub(self.mean).div(self.std)
        x = (x - self.mean) / self.std
        probs = []
        for fmodel in self.fmodels:
            logits = fmodel(x)
            prob = logits.softmax(dim=1)[:, 1]
            probs.append(prob)
        probs = torch.stack(probs, dim=1)
        return probs


def get_sample_batch():
    # return (torch.rand(8, 3, 2048, 1024) * 255).float().cuda()
    img_dir = '/kaggle/tmp/pngs'
    img_names = os.listdir(img_dir)
    imgs = []
    for img_name in img_names[:BATCH_SIZE]:
        img_path = os.path.join(img_dir, img_name)
        img = cv2.imread(img_path)
        imgs.append(img)
    # B H W 3
    batch = np.stack(imgs, axis=0)
    batch = torch.from_numpy(batch)
    batch = batch.permute(0, 3, 1, 2)
    batch = batch.cuda().float()
    return batch


if __name__ == '__main__':
    model = KFoldEnsembleModel(MODEL_NAME, NUM_CLASSES, IN_CHANS, MODEL_CKPTS)
    # print(model)
    model.eval()
    model.cuda()
    sample_batch = get_sample_batch()
    with torch.inference_mode():
        prob = model(sample_batch)
        print(prob.shape, prob)

    # CONVERT TO TENSORRT
    with torch.inference_mode():
        print('START CONVERT')
        model_trt = torch2trt(
            model,
            inputs=[sample_batch],
            input_names=None,
            output_names=None,
            log_level=trt.Logger.ERROR,
            fp16_mode=True,
            max_workspace_size=1 << 32,
            strict_type_constraints=False,
            keep_network=True,
            use_onnx=False,
            default_device_type=trt.DeviceType.GPU,
            dla_core=0,
            gpu_fallback=True,
            device_types={},
            min_shapes=[(1, 3, 2048, 1024)],
            max_shapes=[(BATCH_SIZE, 3, 2048, 1024)],
            opt_shapes=[(BATCH_SIZE, 3, 2048, 1024)],
            onnx_opset=15,
            max_batch_size=BATCH_SIZE,
        )
    torch.save(model_trt.state_dict(), 'fold0_effb4_ep3_th054_trt_fp16.pth')
    print('ALL DONE!')

Writing convert_model.py


## Main code

In [6]:
import warnings

warnings.filterwarnings("ignore")
import os

from metrics import compute_metrics

os.environ['CUDA_MODULE_LOADING'] = 'LAZY'
import ctypes
import gc
import importlib
import multiprocessing as mp
import os
import shutil

import albumentations as A
import cv2
import dicomsdl
import numpy as np
import nvidia.dali as dali
import pandas as pd
import pydicom
import roi_extract
import torch
from albumentations.pytorch.transforms import ToTensorV2
from joblib import Parallel, delayed
from nvidia.dali import types
from nvidia.dali.backend import TensorGPU, TensorListGPU
from torch2trt import TRTModule
from torch.nn import functional as F
from torch.utils.data import DataLoader, Dataset
from tqdm import tqdm
import time

importlib.reload(roi_extract)
import roi_extract

BATCH_SIZE = 4
THRES = 0.52
AUTO_THRES = True
AUTO_THRES_PERCENTILE = 0.97935

J2K_SUID = '1.2.840.10008.1.2.4.90'
J2K_HEADER = b"\x00\x00\x00\x0C"
JLL_SUID = '1.2.840.10008.1.2.4.70'
JLL_HEADER = b"\xff\xd8\xff\xe0"
SUID2HEADER = {J2K_SUID: J2K_HEADER, JLL_SUID: JLL_HEADER}
VOILUT_FUNCS_MAP = {'LINEAR': 0, 'LINEAR_EXACT': 1, 'SIGMOID': 2}
VOILUT_FUNCS_INV_MAP = {v: k for k, v in VOILUT_FUNCS_MAP.items()}
# roi detection
ROI_YOLOX_INPUT_SIZE = [416, 416]
ROI_YOLOX_CONF_THRES = 0.5
ROI_YOLOX_NMS_THRES = 0.9
ROI_YOLOX_HW = [(52, 52), (26, 26), (13, 13)]
ROI_YOLOX_STRIDES = [8, 16, 32]
ROI_AREA_PCT_THRES = 0.04
# model
MODEL_INPUT_SIZE = [2048, 1024]

MODE = 'KAGGLE-TEST'
assert MODE in ['LOCAL-VAL', 'KAGGLE-VAL', 'KAGGLE-TEST']

if MODE == 'KAGGLE-VAL':
    ROI_YOLOX_ENGINE_PATH = '/kaggle/input/kaggle-rsna-ckpts/yolox_nano_bre_416_v2_trt.pth'
    CSV_PATH = '/kaggle/input/kaggle-rsna-ckpts/val_fold_0.csv'
    DCM_ROOT_DIR = '/kaggle/input/rsna-breast-cancer-detection/train_images'
    SAVE_IMG_ROOT_DIR = '/kaggle/tmp/pngs'
    N_CHUNKS = 2
    N_CPUS = 2
    RM_DONE_CHUNK = False
elif MODE == 'KAGGLE-TEST':
    ROI_YOLOX_ENGINE_PATH = '/kaggle/input/kaggle-rsna-ckpts/yolox_nano_bre_416_v2_trt.pth'
    CSV_PATH = '/kaggle/input/rsna-breast-cancer-detection/test.csv'
    DCM_ROOT_DIR = '/kaggle/input/rsna-breast-cancer-detection/test_images'
    SAVE_IMG_ROOT_DIR = '/kaggle/tmp/pngs'
    N_CHUNKS = 2
    N_CPUS = 2
    RM_DONE_CHUNK = True
elif MODE == 'LOCAL-VAL':
    ROI_YOLOX_ENGINE_PATH = '../roi_det/YOLOX/YOLOX_outputs/yolox_nano_bre_416/model_trt.pth'
    CSV_PATH = '../../datasets/cv/v1/val_fold_0.csv'
    DCM_ROOT_DIR = '../../datasets/train_images/'
    SAVE_IMG_ROOT_DIR = './temp_save'
    N_CHUNKS = 2
    N_CPUS = 2
    RM_DONE_CHUNK = False

# DALI patch for INT16 support
################################################################################
DALI2TORCH_TYPES = {
    types.DALIDataType.FLOAT: torch.float32,
    types.DALIDataType.FLOAT64: torch.float64,
    types.DALIDataType.FLOAT16: torch.float16,
    types.DALIDataType.UINT8: torch.uint8,
    types.DALIDataType.INT8: torch.int8,
    types.DALIDataType.UINT16: torch.int16,
    types.DALIDataType.INT16: torch.int16,
    types.DALIDataType.INT32: torch.int32,
    types.DALIDataType.INT64: torch.int64
}


# @TODO: dangerous to copy from UINT16 to INT16 (memory layout?)
# little/big endian ?
# @TODO: faster reuse memory without copying: https://github.com/NVIDIA/DALI/issues/4126
def feed_ndarray(dali_tensor, arr, cuda_stream=None):
    """
    Copy contents of DALI tensor to PyTorch's Tensor.

    Parameters
    ----------
    `dali_tensor` : nvidia.dali.backend.TensorCPU or nvidia.dali.backend.TensorGPU
                    Tensor from which to copy
    `arr` : torch.Tensor
            Destination of the copy
    `cuda_stream` : torch.cuda.Stream, cudaStream_t or any value that can be cast to cudaStream_t.
                    CUDA stream to be used for the copy
                    (if not provided, an internal user stream will be selected)
                    In most cases, using pytorch's current stream is expected (for example,
                    if we are copying to a tensor allocated with torch.zeros(...))
    """
    dali_type = DALI2TORCH_TYPES[dali_tensor.dtype]

    assert dali_type == arr.dtype, (
        "The element type of DALI Tensor/TensorList"
        " doesn't match the element type of the target PyTorch Tensor: "
        "{} vs {}".format(dali_type, arr.dtype))
    assert dali_tensor.shape() == list(arr.size()), \
        ("Shapes do not match: DALI tensor has size {0}, but PyTorch Tensor has size {1}".
            format(dali_tensor.shape(), list(arr.size())))
    cuda_stream = types._raw_cuda_stream(cuda_stream)

    # turn raw int to a c void pointer
    c_type_pointer = ctypes.c_void_p(arr.data_ptr())
    if isinstance(dali_tensor, (TensorGPU, TensorListGPU)):
        stream = None if cuda_stream is None else ctypes.c_void_p(cuda_stream)
        dali_tensor.copy_to_external(c_type_pointer, stream, non_blocking=True)
    else:
        dali_tensor.copy_to_external(c_type_pointer)
    return arr


################################################################################


class PydicomMetadata:

    def __init__(self, ds):
        if "WindowWidth" not in ds or "WindowCenter" not in ds:
            self.window_widths = []
            self.window_centers = []
        else:
            ww = ds['WindowWidth']
            wc = ds['WindowCenter']
            self.window_widths = [float(e)
                                  for e in ww] if ww.VM > 1 else [float(ww.value)]

            self.window_centers = [float(e) for e in wc
                                   ] if wc.VM > 1 else [float(wc.value)]
        
        # if nan --> LINEAR
        self.voilut_func = str(ds.get('VOILUTFunction', 'LINEAR')).upper()
        self.invert = (ds.PhotometricInterpretation == 'MONOCHROME1')
        assert len(self.window_widths) == len(self.window_centers)


class DicomsdlMetadata:

    def __init__(self, ds):
        self.window_widths = ds.WindowWidth
        self.window_centers = ds.WindowCenter
        if self.window_widths is None or self.window_centers is None:
            self.window_widths = []
            self.window_centers = []
        else:
            try:
                if not isinstance(self.window_widths, list):
                    self.window_widths = [self.window_widths]
                self.window_widths = [float(e) for e in self.window_widths]
                if not isinstance(self.window_centers, list):
                    self.window_centers = [self.window_centers]
                self.window_centers = [float(e) for e in self.window_centers]
            except:
                self.window_widths = []
                self.window_centers = []
                
        # if nan --> LINEAR
        self.voilut_func = ds.VOILUTFunction
        if self.voilut_func is None:
            self.voilut_func = 'LINEAR'
        else:
            self.voilut_func = str(self.voilut_func).upper()
        self.invert = (ds.PhotometricInterpretation == 'MONOCHROME1')
        assert len(self.window_widths) == len(self.window_centers)


def min_max_scale(img):
    maxv = img.max()
    minv = img.min()
    if maxv > minv:
        return (img - minv) / (maxv - minv)
    else:
        return img - minv  # ==0


# DEPRECATED: too slow :D
# from Pydicom's source
def apply_windowing_np(arr,
                       window_width=None,
                       window_center=None,
                       voi_func='LINEAR',
                       y_min=0,
                       y_max=255):
    print('WARNING: Deprecated. Using apply_windowing_np_v2() instead.')
    assert window_width > 0
    y_range = y_max - y_min
    # float64 needed (default) or just float32 ?
    # arr = arr.astype(np.float64)
    arr = arr.astype(np.float32)

    if voi_func in ['LINEAR', 'LINEAR_EXACT']:
        # PS3.3 C.11.2.1.2.1 and C.11.2.1.3.2
        if voi_func == 'LINEAR':
            if window_width < 1:
                raise ValueError(
                    "The (0028,1051) Window Width must be greater than or "
                    "equal to 1 for a 'LINEAR' windowing operation")
            window_center -= 0.5
            window_width -= 1
        below = arr <= (window_center - window_width / 2)
        above = arr > (window_center + window_width / 2)
        between = np.logical_and(~below, ~above)

        arr[below] = y_min
        arr[above] = y_max
        if between.any():
            arr[between] = ((
                (arr[between] - window_center) / window_width + 0.5) * y_range
                            + y_min)
    elif voi_func == 'SIGMOID':
        arr = y_range / (1 +
                         np.exp(-4 *
                                (arr - window_center) / window_width)) + y_min
    else:
        raise ValueError(
            f"Unsupported (0028,1056) VOI LUT Function value '{voi_func}'")
    return arr


def apply_windowing_np_v2(arr,
                          window_width=None,
                          window_center=None,
                          voi_func='LINEAR',
                          y_min=0,
                          y_max=255):
    assert window_width > 0
    y_range = y_max - y_min
    # float64 needed (default) or just float32 ?
    # arr = arr.astype(np.float64)
    arr = arr.astype(np.float32)

    if voi_func == 'LINEAR' or voi_func == 'LINEAR_EXACT':
        # PS3.3 C.11.2.1.2.1 and C.11.2.1.3.2
        if voi_func == 'LINEAR':
            if window_width < 1:
                raise ValueError(
                    "The (0028,1051) Window Width must be greater than or "
                    "equal to 1 for a 'LINEAR' windowing operation")
            window_center -= 0.5
            window_width -= 1

        # simple trick to improve speed
        s = y_range / window_width
        b = (-window_center / window_width + 0.5) * y_range + y_min
        arr = arr * s + b
        arr = np.clip(arr, y_min, y_max)

    elif voi_func == 'SIGMOID':
        # simple trick to improve speed
        s = -4 / window_width
        arr = y_range / (1 + np.exp((arr - window_center) * s)) + y_min
    else:
        raise ValueError(
            f"Unsupported (0028,1056) VOI LUT Function value '{voi_func}'")
    return arr


def apply_windowing_torch(arr,
                          window_width=None,
                          window_center=None,
                          voi_func='LINEAR',
                          y_min=0,
                          y_max=255):
    assert window_width > 0
    y_range = y_max - y_min
    # float64 needed (default) or just float32 ?
    # arr = arr.double()
    arr = arr.float()

    if voi_func == 'LINEAR' or voi_func == 'LINEAR_EXACT':
        # PS3.3 C.11.2.1.2.1 and C.11.2.1.3.2
        if voi_func == 'LINEAR':
            if window_width < 1:
                raise ValueError(
                    "The (0028,1051) Window Width must be greater than or "
                    "equal to 1 for a 'LINEAR' windowing operation")
            window_center -= 0.5
            window_width -= 1

        # simple trick to improve speed
        s = y_range / window_width
        b = (-window_center / window_width + 0.5) * y_range + y_min
        arr = arr * s + b
        arr = torch.clamp(arr, y_min, y_max)

    elif voi_func == 'SIGMOID':
        # simple trick to improve speed
        s = -4 / window_width
        arr = y_range / (1 + torch.exp((arr - window_center) * s)) + y_min
    else:
        raise ValueError(
            f"Unsupported (0028,1056) VOI LUT Function value '{voi_func}'")
    return arr


def resize_and_pad(img, input_size=MODEL_INPUT_SIZE):
    input_h, input_w = input_size
    ori_h, ori_w = img.shape[:2]
    ratio = min(input_h / ori_h, input_w / ori_w)
    # resize
    img = F.interpolate(img.view(1, 1, ori_h, ori_w),
                        mode="bilinear",
                        scale_factor=ratio,
                        recompute_scale_factor=True)[0, 0]
    # padding
    padded_img = torch.zeros((input_h, input_w),
                             dtype=img.dtype,
                             device='cuda')
    cur_h, cur_w = img.shape
    y_start = (input_h - cur_h) // 2
    x_start = (input_w - cur_w) // 2
    padded_img[y_start:y_start + cur_h, x_start: x_start + cur_w] = img
    padded_img = padded_img.unsqueeze(-1).expand(-1, -1, 3)
    return padded_img


def save_img_to_file(save_path, img, backend='cv2'):
    file_ext = os.path.basename(save_path).split('.')[-1]
    if backend == 'cv2':
        if img.dtype == np.uint16:
            # https://docs.opencv.org/3.4/d4/da8/group__imgcodecs.html#gabbc7ef1aa2edfaa87772f1202d67e0ce
            assert file_ext in ['png', 'jp2', 'tiff', 'tif']
            cv2.imwrite(save_path, img)
        elif img.dtype == np.uint8:
            cv2.imwrite(save_path, img)
        else:
            raise ValueError(
                '`cv2` backend only support uint8 or uint16 images.')
    elif backend == 'np':
        assert file_ext == 'npy'
        np.save(save_path, img)
    else:
        raise ValueError(f'Unsupported backend `{backend}`.')


def load_img_from_file(img_path, backend='cv2'):
    if backend == 'cv2':
        return cv2.imread(img_path, cv2.IMREAD_ANYDEPTH)
    elif backend == 'np':
        return np.load(img_path)
    else:
        raise ValueError()


class _JStreamExternalSource:

    def __init__(self, dcm_paths, batch_size=1):
        self.dcm_paths = dcm_paths
        self.len = len(dcm_paths)
        self.batch_size = batch_size

    def __call__(self, batch_info):
        idx = batch_info.iteration
        # print('IDX:', batch_info.iteration, batch_info.epoch_idx)
        start = idx * self.batch_size
        end = min(self.len, start + self.batch_size)
        if end <= start:
            raise StopIteration()

        batch_dcm_paths = self.dcm_paths[start:end]
        j_streams = []
        inverts = []
        windowing_params = []
        voilut_funcs = []

        for dcm_path in batch_dcm_paths:
            ds = pydicom.dcmread(dcm_path)
            pixel_data = ds.PixelData
            offset = pixel_data.find(
                SUID2HEADER[ds.file_meta.TransferSyntaxUID])
            
#             for ss in [10152, 10315, 10342, 1036, 10432, 10439, 12678]:
#                 if str(ss) in dcm_path:
#                     offset += 2
#             if idx % 5 == 4:
#                 offset += 2
                    
            j_stream = np.array(bytearray(pixel_data[offset:]), np.uint8)
            invert = (ds.PhotometricInterpretation == 'MONOCHROME1')
            meta = PydicomMetadata(ds)
            windowing_param = np.array(
                [meta.window_centers, meta.window_widths], np.float16)
            voilut_func = VOILUT_FUNCS_MAP[meta.voilut_func]
            j_streams.append(j_stream)
            inverts.append(invert)
            windowing_params.append(windowing_param)
            voilut_funcs.append(voilut_func)
#         return j_streams, inverts, windowing_params, voilut_funcs
        return j_streams, np.array(inverts, dtype=np.bool_), \
            windowing_params, np.array(voilut_funcs, dtype=np.uint8)


@dali.pipeline_def
def _dali_pipeline(eii):
    jpeg, invert, windowing_param, voilut_func = dali.fn.external_source(
        source=eii,
        num_outputs=4,
        dtype=[
            dali.types.UINT8, dali.types.BOOL, dali.types.FLOAT16,
            dali.types.UINT8
        ],
        batch=True,
        batch_info=True,
        parallel=True)
    ori_img = dali.fn.experimental.decoders.image(
        jpeg,
        device='mixed',
        output_type=dali.types.ANY_DATA,
        dtype=dali.types.UINT16)
    return ori_img, invert, windowing_param, voilut_func


def decode_crop_save_dali(dcm_paths,
                          save_paths,
                          save_backend='cv2',
                          batch_size=1,
                          num_threads=1,
                          py_num_workers=1,
                          py_start_method='fork',
                          device_id=0):

    assert len(dcm_paths) == len(save_paths)
    assert save_backend in ['cv2', 'np']
    num_dcms = len(dcm_paths)

    # dali to process with chunk in-memory
    external_source = _JStreamExternalSource(dcm_paths, batch_size=batch_size)
    pipe = _dali_pipeline(
        external_source,
        py_num_workers=py_num_workers,
        py_start_method=py_start_method,
        batch_size=batch_size,
        num_threads=num_threads,
        device_id=device_id,
        debug=False,
    )
    pipe.build()

    roi_extractor = roi_extract.RoiExtractor(engine_path=ROI_YOLOX_ENGINE_PATH,
                                             input_size=ROI_YOLOX_INPUT_SIZE,
                                             num_classes=1,
                                             conf_thres=ROI_YOLOX_CONF_THRES,
                                             nms_thres=ROI_YOLOX_NMS_THRES,
                                             class_agnostic=False,
                                             area_pct_thres=ROI_AREA_PCT_THRES,
                                             hw=ROI_YOLOX_HW,
                                             strides=ROI_YOLOX_STRIDES,
                                             exp=None)
    print('ROI extractor (YOLOX) loaded!')

    num_batchs = num_dcms // batch_size
    last_batch_size = batch_size
    if num_dcms % batch_size > 0:
        num_batchs += 1
        last_batch_size = num_dcms % batch_size

    cur_idx = -1
    for _batch_idx in tqdm(range(num_batchs)):
        try:
            outs = pipe.run()
        except Exception as e:
#             print('DALI exception occur:', e)
            print(
                f'Exception: One of {dcm_paths[_batch_idx * batch_size: (_batch_idx + 1) * batch_size]} can not be decoded.'
            )
            # ignore this batch and re-build pipeline
            if _batch_idx < num_batchs - 1:
                cur_idx += batch_size
                del external_source, pipe
                gc.collect()
                torch.cuda.empty_cache()
                external_source = _JStreamExternalSource(
                    dcm_paths[(_batch_idx + 1) * batch_size:],
                    batch_size=batch_size)
                pipe = _dali_pipeline(
                    external_source,
                    py_num_workers=py_num_workers,
                    py_start_method=py_start_method,
                    batch_size=batch_size,
                    num_threads=num_threads,
                    device_id=device_id,
                    debug=False,
                )
                pipe.build()
            else:
                cur_idx += last_batch_size
            continue

        imgs = outs[0]
        inverts = outs[1]
        windowing_params = outs[2]
        voilut_funcs = outs[3]
        for j in range(len(inverts)):
            cur_idx += 1
            save_path = save_paths[cur_idx]
            img_dali = imgs[j]
            img_torch = torch.empty(img_dali.shape(),
                                    dtype=torch.int16,
                                    device='cuda')
            feed_ndarray(img_dali,
                         img_torch,
                         cuda_stream=torch.cuda.current_stream(device=0))
            # @TODO: test whether copy uint16 to int16 pointer is safe in this case
            if 0:
                img_np = img_dali.as_cpu().squeeze(-1)  # uint16
                print(type(img_np), img_np.shape)
                img_np = torch.from_numpy(img_np, dtype=torch.int16)
                diff = torch.max(torch.abs(img_np - img_torch))
                assert diff == 0, f'{img_torch.shape}, {img_np.shape}, {diff}'

            invert = inverts.at(j).item()
            windowing_param = windowing_params.at(j)
            voilut_func = voilut_funcs.at(j).item()
            voilut_func = VOILUT_FUNCS_INV_MAP[voilut_func]

            # YOLOX for ROI extraction
            # @TODO: optimize and sync training vs inference pipeline:
            # @current: uint16 --> min_max_norm --> uint8 --> resize (cv2) --> padding
            # @faster?: uint16 --> resize (torch interpolate) --> min_max_norm --> uint8 --> padding
            # @fastest?: uint16 --> VOILUT --> resize (torch interpolate) --> uint8 --> padding
            img_yolox = min_max_scale(img_torch)
            img_yolox = (img_yolox * 255)  # float32
            if invert:
                img_yolox = 255 - img_yolox
            # YOLOX infer
            # who know if exception happen in hidden test ?
            try:
                xyxy, _area_pct, _conf = roi_extractor.detect_single(img_yolox)
                if xyxy is not None:
                    x0, y0, x1, y1 = xyxy
                    crop = img_torch[y0:y1, x0:x1]
                else:
                    crop = img_torch
            except:
                print('ROI extract exception!')
                crop = img_torch

            # apply windowing
            if windowing_param.shape[1] != 0:
                default_window_center = windowing_param[0, 0]
                default_window_width = windowing_param[1, 0]
                crop = apply_windowing_torch(crop,
                                             window_width=default_window_width,
                                             window_center=default_window_center,
                                             voi_func=voilut_func,
                                             y_min=0,
                                             y_max=255)
            # if no window center/width in dcm file
            # do simple min-max scaling
            else:
                print('No windowing param!')
                crop = min_max_scale(crop)
                crop = crop * 255
            if invert:
                crop = 255 - crop
            crop = resize_and_pad(crop, MODEL_INPUT_SIZE)
            crop = crop.to(torch.uint8)
            crop = crop.cpu().numpy()
            save_img_to_file(save_path, crop, backend=save_backend)

#     assert cur_idx == len(
#         save_paths) - 1, f'{cur_idx} != {len(save_paths) - 1}'

    try:
        del external_source, pipe, roi_extractor
    except:
        pass
    gc.collect()
    torch.cuda.empty_cache()
    return


def decode_and_save_dali_parallel(
        dcm_paths,
        save_paths,
        save_backend='cv2',
        batch_size=1,
        num_threads=1,
        py_num_workers=1,
        py_start_method='fork',
        device_id=0,
        parallel_n_jobs=2,
        parallel_n_chunks = 4,
        parallel_backend='joblib',  # joblib or multiprocessing
        joblib_backend='loky'):
    assert parallel_backend in ['joblib', 'multiprocessing']
    assert joblib_backend in ['threading', 'multiprocessing', 'loky']
    # py_num_workers > 0 means using multiprocessing worker
    # 'fork' multiprocessing after CUDA init is not work (we must use 'spawn' instead)
    # since our pipeline can be re-build (when a dicom can't be decoded on GPU),
    # 2 options:
    #       (py_num_workers = 0, py_start_method=?)
    #       (py_num_workers > 0, py_start_method = 'spawn')
    assert not (py_num_workers > 0 and py_start_method == 'fork')

    if parallel_n_jobs == 1:
        print('No parralel. Starting the tasks within current process.')
        return decode_crop_save_dali(dcm_paths,
                                     save_paths,
                                     save_backend=save_backend,
                                     batch_size=batch_size,
                                     num_threads=num_threads,
                                     py_num_workers=py_num_workers,
                                     py_start_method=py_start_method,
                                     device_id=device_id)
    else:
        num_samples = len(dcm_paths)
        num_samples_per_chunk = num_samples // parallel_n_chunks
        if num_samples % parallel_n_chunks > 0:
            num_samples_per_chunk += 1
        starts = [num_samples_per_chunk * i for i in range(parallel_n_chunks)]
        ends = [
            min(start + num_samples_per_chunk, num_samples)
            for start in starts
        ]
        if isinstance(device_id, list):
            assert len(device_id) == parallel_n_chunks
        elif isinstance(device_id, int):
            device_id = [device_id] * parallel_n_chunks

        print(
            f'Starting {parallel_n_jobs} jobs with backend `{parallel_backend}`, {parallel_n_chunks} chunks ...'
        )
        if parallel_backend == 'joblib':
            _ = Parallel(n_jobs=parallel_n_jobs, backend=joblib_backend)(
                delayed(decode_crop_save_dali)(
                    dcm_paths[start:end],
                    save_paths[start:end],
                    save_backend=save_backend,
                    batch_size=batch_size,
                    num_threads=num_threads,
                    py_num_workers=py_num_workers,  # ram_v3
                    py_start_method=py_start_method,
                    device_id=worker_device_id,
                ) for start, end, worker_device_id in zip(
                    starts, ends, device_id))
        else:  # manually start multiprocessing's processes
            workers = []
            daemon = False if py_num_workers > 0 else True
            for i in range(parallel_n_jobs):
                start = starts[i]
                end = ends[i]
                worker_device_id = device_id[i]
                worker = mp.Process(group=None,
                                    target=decode_crop_save_dali,
                                    args=(
                                        dcm_paths[start:end],
                                        save_paths[start:end],
                                    ),
                                    kwargs={
                                        'save_backend': save_backend,
                                        'batch_size': batch_size,
                                        'num_threads': num_threads,
                                        'py_num_workers': py_num_workers,
                                        'py_start_method': py_start_method,
                                        'device_id': worker_device_id,
                                    },
                                    daemon=daemon)
                workers.append(worker)
            for worker in workers:
                worker.start()
            for worker in workers:
                worker.join()
    return


def _single_decode_crop_save_sdl(roi_extractor,
                                 dcm_path,
                                 save_path,
                                 save_backend='cv2',
                                 index=0):
    dcm = dicomsdl.open(dcm_path)
    meta = DicomsdlMetadata(dcm)
    info = dcm.getPixelDataInfo()
    if info['SamplesPerPixel'] != 1:
        raise RuntimeError('SamplesPerPixel != 1')
    else:
        shape = [info['Rows'], info['Cols']]

    # ori_dtype = info['dtype']
    # img = np.empty(shape, dtype=ori_dtype)
    # assert img.dtype == np.uint16, f'{img.dtype}'
    # dcm.copyFrameData(index, img)
    # img_torch = torch.from_numpy(img.astype(np.int16)).cuda()

    img = np.empty(shape, dtype=np.int16)
    dcm.copyFrameData(index, img)
    img_torch = torch.from_numpy(img).cuda()  # int16

    # YOLOX for ROI extraction
    # @TODO: optimize and sync training vs inference pipeline:
    # @current: uint16 --> min_max_norm --> uint8 --> resize (cv2) --> padding
    # @faster?: uint16 --> resize (torch interpolate) --> min_max_norm --> uint8 --> padding
    # @fastest?: uint16 --> VOILUT --> resize (torch interpolate) --> uint8 --> padding
    img_yolox = min_max_scale(img_torch)
    img_yolox = (img_yolox * 255)  # float32
    # @TODO: subtract on large array --> should move after F.interpolate()
    if meta.invert:
        img_yolox = 255 - img_yolox
    # YOLOX infer
    try:
        xyxy, _area_pct, _conf = roi_extractor.detect_single(img_yolox)
        if xyxy is not None:
            x0, y0, x1, y1 = xyxy
            crop = img_torch[y0:y1, x0:x1]
        else:
            crop = img_torch
    except:
        print('ROI extract exception!')
        crop = img_torch

    # apply voi lut
    if meta.window_widths:
        crop = apply_windowing_torch(crop,
                                     window_width=meta.window_widths[0],
                                     window_center=meta.window_centers[0],
                                     voi_func=meta.voilut_func,
                                     y_min=0,
                                     y_max=255)
    else:
        print('No windowing param!')
        crop = min_max_scale(crop)
        crop = crop * 255
        
    if meta.invert:
        crop = 255 - crop
    crop = resize_and_pad(crop, MODEL_INPUT_SIZE)
    crop = crop.to(torch.uint8)
    crop = crop.cpu().numpy()
    save_img_to_file(save_path, crop, backend=save_backend)


def decode_crop_save_sdl(dcm_paths, save_paths, save_backend='cv2'):
    assert len(dcm_paths) == len(save_paths)
    roi_detector = roi_extract.RoiExtractor(engine_path=ROI_YOLOX_ENGINE_PATH,
                                            input_size=ROI_YOLOX_INPUT_SIZE,
                                            num_classes=1,
                                            conf_thres=ROI_YOLOX_CONF_THRES,
                                            nms_thres=ROI_YOLOX_NMS_THRES,
                                            class_agnostic=False,
                                            area_pct_thres=ROI_AREA_PCT_THRES,
                                            hw=ROI_YOLOX_HW,
                                            strides=ROI_YOLOX_STRIDES,
                                            exp=None)
    print('ROI extractor (YOLOX) loaded!')
    for i in tqdm(range(len(dcm_paths))):
        _single_decode_crop_save_sdl(roi_detector, dcm_paths[i], save_paths[i],
                                     save_backend)

    del roi_detector
    gc.collect()
    torch.cuda.empty_cache()
    return


def decode_crop_save_sdl_parallel(dcm_paths,
                                  save_paths,
                                  save_backend='cv2',
                                  parallel_n_jobs=2,
                                  parallel_n_chunks = 4,
                                  joblib_backend='loky'):
    assert len(dcm_paths) == len(save_paths)
    if parallel_n_jobs == 1:
        print('No parralel. Starting the tasks within current process.')
        return decode_crop_save_sdl(dcm_paths, save_paths, save_backend)
    else:
        num_samples = len(dcm_paths)
        num_samples_per_chunk = num_samples // parallel_n_chunks
        if num_samples % parallel_n_chunks > 0:
            num_samples_per_chunk += 1
        starts = [num_samples_per_chunk * i for i in range(parallel_n_chunks)]
        ends = [
            min(start + num_samples_per_chunk, num_samples)
            for start in starts
        ]

        print(
            f'Starting {parallel_n_jobs} jobs with backend `{joblib_backend}`, {parallel_n_chunks} chunks...'
        )
        _ = Parallel(n_jobs=parallel_n_jobs, backend=joblib_backend)(
            delayed(decode_crop_save_sdl)(dcm_paths[start:end],
                                          save_paths[start:end], save_backend)
            for start, end in zip(starts, ends))


class ValTransform:

    def __init__(self):
        self.transform_fn = A.Compose([ToTensorV2(transpose_mask=True)])

    def __call__(self, img):
        return self.transform_fn(image=img)['image']


class RSNADataset(Dataset):

    def __init__(self, df, img_root_dir, transform_fn=None):
        self.img_paths = []
        self.transform_fn = transform_fn
        self.df = df
        for i in tqdm(range(len(df))):
            patient_id = df.at[i, 'patient_id']
            image_id = df.at[i, 'image_id']
            img_name = f'{patient_id}@{image_id}.png'
            img_path = os.path.join(img_root_dir, img_name)
            self.img_paths.append(img_path)
        print(f'Done loading dataset with {len(self.img_paths)} samples.')

    def __len__(self):
        return len(self.img_paths)

    def __getitem__(self, idx):
        img_path = self.img_paths[idx]
        img = cv2.imread(img_path)
        if img is None:
            print('ERROR:', img_path)
        if self.transform_fn:
            img = self.transform_fn(img)
        return img

    def get_df(self):
        return self.df


######################################################
# MAIN CODE

if MODE == 'KAGGLE-TEST':
    global_df = pd.read_csv(CSV_PATH)
else:
    global_df = pd.read_csv(CSV_PATH)[:1000]
all_patients = list(global_df.patient_id.unique())
num_patients = len(all_patients)

num_patients_per_chunk = num_patients // N_CHUNKS + 1
chunk_patients = [
    all_patients[num_patients_per_chunk * i:num_patients_per_chunk * (i + 1)]
    for i in range(N_CHUNKS)
]
print(f'PATIENT CHUNKS: {[len(c) for c in chunk_patients]}')

pred_dfs = []
for chunk_idx, chunk_patients in enumerate(chunk_patients):
    os.makedirs(SAVE_IMG_ROOT_DIR, exist_ok=True)
    df = global_df[global_df.patient_id.isin(chunk_patients)].reset_index(
        drop=True)
    print(
        f'Processing chunk {chunk_idx} with {len(chunk_patients)} patients, {len(df)} images'
    )
    if len(df) == 0:
        continue
    dcm_paths = []
    save_paths = []
    for i in range(len(df)):
        patient_id = df.at[i, 'patient_id']
        image_id = df.at[i, 'image_id']
        dcm_path = os.path.join(DCM_ROOT_DIR, str(patient_id),
                                f'{image_id}.dcm')
        save_path = os.path.join(SAVE_IMG_ROOT_DIR,
                                 f'{patient_id}@{image_id}.png')

        # if os.path.isfile(save_path):
        #     continue
        # else:
        #     dcm_paths.append(dcm_path)
        #     save_paths.append(save_path)
        dcm_paths.append(dcm_path)
        save_paths.append(save_path)

    if 1:
        t0 = time.time()
        # try to decode all with DALI
        decode_and_save_dali_parallel(
            dcm_paths,
            save_paths,
            save_backend='cv2',
            batch_size=1,
            num_threads=1,
            py_num_workers=0,
            py_start_method='fork',
            device_id=0,
            parallel_n_jobs=N_CPUS * 2,
            parallel_n_chunks = N_CPUS * 2,
            parallel_backend='joblib',  # joblib or multiprocessing
            joblib_backend='loky')
        gc.collect()
        torch.cuda.empty_cache()
        t1 = time.time()
        print(f'DALI done in {t1 - t0} sec')


        # CPU decode all others (exceptions) with dicomsdl
        done_img_names = os.listdir(SAVE_IMG_ROOT_DIR)
        save_img_names = [os.path.basename(p) for p in save_paths]
        remain_img_names = list(set(save_img_names) - set(done_img_names))
        remain_img_paths = [
            os.path.join(SAVE_IMG_ROOT_DIR, name) for name in remain_img_names
        ]
        remain_dcm_paths = []
        for name in remain_img_names:
            patient_id, image_id = os.path.basename(name).split('.')[0].split(
                '@')
            remain_dcm_paths.append(
                os.path.join(DCM_ROOT_DIR, patient_id, f'{image_id}.dcm'))
        num_remain = len(remain_dcm_paths)
        print(f'Number of undecoded files: {num_remain}')
#         print(f'Remains: {remain_img_names}')
        if num_remain > 0:
            # 16 or just any > 0 number
            if num_remain > 32 * N_CPUS:
                sdl_n_jobs = N_CPUS * 2
                sdl_n_chunks = N_CPUS * 2,
            else:
                sdl_n_jobs = 1
                sdl_n_chunks = 1
            decode_crop_save_sdl_parallel(remain_dcm_paths,
                                          remain_img_paths,
                                          save_backend='cv2',
                                          parallel_n_jobs=sdl_n_jobs,
                                          parallel_n_chunks = sdl_n_chunks,
                                          joblib_backend='loky')
            gc.collect()
            torch.cuda.empty_cache()
        else:
            print('No remain files to decode.')
        t2 = time.time()
        print(f'SDL done in { t2 - t1} sec')
        print(f'TOTAL DECODING TIME: {t2 - t0} sec')

    dataset = RSNADataset(df, SAVE_IMG_ROOT_DIR, transform_fn=ValTransform())
    dataloader = DataLoader(
        dataset,
        batch_size=BATCH_SIZE,
        shuffle=False,
        num_workers=2,
        pin_memory=True,
        drop_last=False,
    )

#     wrap_autocast = False
#     model = TRTModule()
#     model.load_state_dict(torch.load('/kaggle/input/kaggle-rsna-ckpts/fold0_convnext-small_exp4_ratio8_ep11_trt_fp32.pth'))

    ############# TORCH MODEL ############
    wrap_autocast = True
#     MODEL_NAME = 'tf_efficientnet_b4.ns_jft_in1k'
    MODEL_NAME = 'convnext_small.fb_in22k_ft_in1k_384'
    MODEL_CKPTS = [
        '/kaggle/input/kaggle-rsna-ckpts/fold0_convnext-small_exp4_ratio8_ep11.pth.tar',
    ]
    NUM_CLASSES = 2
    IN_CHANS = 3
    MEAN = np.array([0.485, 0.456, 0.406]) * 255
    STD = np.array([0.229, 0.224, 0.225]) * 255
    GLOBAL_POOL = 'avg'
    from convert_model import KFoldEnsembleModel
    from functools import partial
    AMP_DTYPE = torch.float16
    # AMP_DTYPE = torch.bfloat16
    amp_autocast = partial(torch.autocast, device_type='cuda', dtype=AMP_DTYPE)
    model = KFoldEnsembleModel(MODEL_NAME, NUM_CLASSES, IN_CHANS, MODEL_CKPTS)
    model.eval()
    model.cuda()
    ############ TORCH MODEL ################

    all_probs = []
    with torch.inference_mode():
        for batch in tqdm(dataloader):
            batch = batch.cuda().float()
            if wrap_autocast:
                with amp_autocast():
                    probs = model(batch)
            else:
                probs = model(batch)
            probs = probs.cpu().numpy()
            all_probs.append(probs)
    
    # N * num_models
    all_probs = np.concatenate(all_probs, axis=0)
    all_probs = np.nan_to_num(all_probs, nan=0.0, posinf=None, neginf=None)
    all_probs = all_probs.mean(axis=-1)
    assert all_probs.shape[0] == len(df)

    df['preds'] = all_probs
    pred_dfs.append(df)

    print(f'DONE CHUNK {chunk_idx} with {len(df)} samples')
    del model
    gc.collect()
    torch.cuda.empty_cache()
    if RM_DONE_CHUNK:
        shutil.rmtree(SAVE_IMG_ROOT_DIR)
        print(f'Removed save image directory {SAVE_IMG_ROOT_DIR}')
    print('-----------------------------\n\n')

    
pred_df = pd.concat(pred_dfs).reset_index(drop=True)
if 'prediction_id' not in pred_df.columns:
    pred_df['prediction_id'] = pred_df.apply(lambda row: str(row.patient_id) + '_' + row.laterality, axis = 1)
submit_df = pred_df[['prediction_id', 'preds']]
submit_df = pred_df.groupby('prediction_id').mean()
if AUTO_THRES:
    thres = np.quantile(submit_df['preds'].values, AUTO_THRES_PERCENTILE)
else:
    thres = THRES
submit_df['cancer'] = (submit_df['preds'].values > thres).astype(int)
submit_df = submit_df['cancer']
submit_df.to_csv('submission.csv')
    
if MODE == 'KAGGLE-TEST':
    pass
else:
    pred_df['targets'] = pred_df['cancer']
    pred_df.to_csv('prediction.csv')
    metrics = compute_metrics(pred_df,
                              plot_save_path='metric_plot.png',
                              thres_range=(0, 1, 0.01),
                              sort_by='pfbeta',
                              additional_info=True)
    print('METRICS:', metrics)

PATIENT CHUNKS: [1, 0]
Processing chunk 0 with 1 patients, 4 images
Starting 4 jobs with backend `joblib`, 4 chunks ...
[02/16/2023-22:24:49] [TRT] [W] TensorRT was linked against cuDNN 8.4.1 but loaded cuDNN 8.0.5
[02/16/2023-22:24:49] [TRT] [W] TensorRT was linked against cuDNN 8.4.1 but loaded cuDNN 8.0.5
[02/16/2023-22:24:49] [TRT] [W] TensorRT was linked against cuDNN 8.4.1 but loaded cuDNN 8.0.5
[02/16/2023-22:24:49] [TRT] [W] TensorRT was linked against cuDNN 8.4.1 but loaded cuDNN 8.0.5
[02/16/2023-22:24:49] [TRT] [W] TensorRT was linked against cuDNN 8.4.1 but loaded cuDNN 8.0.5
[02/16/2023-22:24:49] [TRT] [W] TensorRT was linked against cuDNN 8.4.1 but loaded cuDNN 8.0.5
[02/16/2023-22:24:49] [TRT] [W] TensorRT was linked against cuDNN 8.4.1 but loaded cuDNN 8.0.5
[02/16/2023-22:24:49] [TRT] [W] TensorRT was linked against cuDNN 8.4.1 but loaded cuDNN 8.0.5
ROI extractor (YOLOX) loaded!
ROI extractor (YOLOX) loaded!
ROI extractor (YOLOX) loaded!
ROI extractor (YOLOX) loaded!


100%|██████████| 1/1 [00:08<00:00,  8.54s/it]

DALI done in 22.940833806991577 sec
Number of undecoded files: 0
No remain files to decode.
SDL done in 0.0005478858947753906 sec
TOTAL DECODING TIME: 22.941381692886353 sec


100%|██████████| 4/4 [00:00<00:00, 8469.06it/s]

Done loading dataset with 4 samples.


Data config: {'input_size': (3, 384, 384), 'interpolation': 'bicubic', 'mean': (0.485, 0.456, 0.406), 'std': (0.229, 0.224, 0.225), 'crop_pct': 1.0, 'crop_mode': 'squash'}


100%|██████████| 1/1 [00:02<00:00,  2.70s/it]


DONE CHUNK 0 with 4 samples
Removed save image directory /kaggle/tmp/pngs
-----------------------------


Processing chunk 1 with 0 patients, 0 images


In [7]:
# !ls /kaggle/tmp/pngs  | wc -l
# !du -sh /kaggle/tmp/pngs

In [8]:
# if MODE != 'KAGGLE-TEST':    
#     from IPython.display import Image, display as idisplay
#     import PIL
#     from PIL import Image


#     def longest_resize(img, resize = 416):
#         h, w = img.shape[:2]
#         r = resize / max(h, w)
#         new_h, new_w = int(r * h), int(r * w)
#         img = cv2.resize(img, (new_w, new_h))
#         return img

#     def display(img, resize = None):
#     #     img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
#         if resize:
#             img = longest_resize(img, resize)
#         idisplay(Image.fromarray(img))
#         print('\n--------------------------\n')

#     names = os.listdir(SAVE_IMG_ROOT_DIR)
#     img_paths = [os.path.join(SAVE_IMG_ROOT_DIR, name) for name in names]

#     for img_path in img_paths[:20]:
#         print(img_path)
#         img = cv2.imread(img_path)
#         display(img, resize = 640)

In [9]:
# try:
#     import onnx_graphsurgeon
# except:
#     !pip install onnxruntime-gpu
#     !pip install onnx_graphsurgeon --index-url https://pypi.ngc.nvidia.com

In [10]:
# from torch2trt import TRTModule

# model_torch = KFoldEnsembleModel(MODEL_NAME, NUM_CLASSES, IN_CHANS, MODEL_CKPTS)
# model_torch.eval()
# model_torch.cuda()

# model_trt = TRTModule()
# model_trt.load_state_dict(torch.load('fold0_effb4_ep3_th054_trt_fp16.pth'))

# with torch.inference_mode():
#     for i in range(100):
#         x = (torch.rand(4, 3, 2048, 1024).float() * 255).cuda()
#         prob_torch = model_torch(x)
#         prob_trt = model_trt(x)
#         diff = torch.max(torch.abs(prob_torch - prob_trt))
#         print(prob_torch)
#         print(prob_trt)
#         print(prob_torch.shape, prob_trt.shape, diff)
#         print('----------')

In [11]:
# !nvidia-smi

In [12]:
# try:
#     del model, sample_inp
# except:
#     pass
# gc.collect()
# torch.cuda.empty_cache()